<a href="https://colab.research.google.com/github/ZaeemAkbarKhan786/MS_ProgrammingForAI/blob/main/bert_fine_tuning_mr_google_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

-------------------------------------------------------------

# **Name:** Zaeem Akbar Khan
# **Registration #:** FA24-RAI-019
# **Course:** Programming for AI
# **Assignment #:** 03

-------------------------------------------------------------

**Base Paper Name:** Bert-Enhanced Text Graph Neural Network for Classification

# **Selected Paper**
✔ Recent Research (2020–2024)

✔ Reputable Journal (W Category)

✔ BERT-Based Model for Text Classification

✔ Clear Description of Dataset, Problem, and Methodology


*   The problem statement (text classification).
*   Datasets (MR, SST-2, R8, Ohsumed).
*   Methodology, including graph construction, BERT-based feature extraction, and feature aggregation.

✔ Code Availability

## **Selected Dataset:**
**MR**

**Link:** https://www.kaggle.com/datasets/weipengfei/ohr8r52

# **Selected Paper links:**
**PubMed:** https://pubmed.ncbi.nlm.nih.gov/34828233/

**ResearchGate:** https://www.researchgate.net/publication/356388348_Bert-Enhanced_Text_Graph_Neural_Network_for_Classification

**Semantic Scholar:** https://www.semanticscholar.org/paper/Bert-Enhanced-Text-Graph-Neural-Network-for-Yang-Cui/108eea36fc140fc0aeb2b6024925b28a7125c2ca

# **Base Paper Code:**
**Link:** https://github.com/pingpingand/BEGNN

**Installing Dependencies**

In [ ]:
!pip install -U "tensorflow-text==2.13.*"

**Importing Required Libraries**

In [ ]:
!pip install --quiet tensorflow_text
import re
import nltk
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')
stopwords.words('english')
exclude = string.punctuation

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


**Loading the Dataset**

In [ ]:
tempdf = pd.read_csv('/content/MR.csv')

**Checking Dataset Shape**

In [ ]:
tempdf.shape

(1000, 3)

**Viewing the Dataset Head**

In [ ]:
tempdf.head()

,Unnamed: 0,review,sentiment
0,0,i'm convinced i could keep a family of five b...,negative
1,1,davis is so enamored of her own creation that...,negative
2,2,it 's hard to imagine that even very small ch...,negative
3,3,"somewhere in the middle , the film compels , ...",negative
4,4,"a muckraking job , the cinematic equivalent o...",positive


**Dropping Unnecessary Columns**

In [ ]:
tempdf.drop("Unnamed: 0", axis = 1, inplace= True)

**Selecting a Subset of Data**

In [ ]:
df = tempdf.iloc[:100]

**Viewing the Subset Data Head**

In [ ]:
df.head()

,review,sentiment
0,i'm convinced i could keep a family of five b...,negative
1,davis is so enamored of her own creation that...,negative
2,it 's hard to imagine that even very small ch...,negative
3,"somewhere in the middle , the film compels , ...",negative
4,"a muckraking job , the cinematic equivalent o...",positive


**Text PreProcessing Functions**

In [ ]:
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

def remove_punc(text):
    return text.translate(str.maketrans('', '', exclude))

**Applying Preprocessing Steps**

In [ ]:
df['review'] = df['review'].str.lower()

df['review'] = df['review'].apply(remove_html_tags)

df['review'] = df['review'].apply(remove_url)

df['review'] = df['review'].apply(remove_punc)

<ipython-input-10-910011608a88>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].str.lower()
<ipython-input-10-910011608a88>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_html_tags)
<ipython-input-10-910011608a88>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

**Viewing Preprocessed Data**

In [ ]:
df.head()

,review,sentiment
0,im convinced i could keep a family of five bl...,negative
1,davis is so enamored of her own creation that...,negative
2,it s hard to imagine that even very small chi...,negative
3,somewhere in the middle the film compels as...,negative
4,a muckraking job the cinematic equivalent of...,positive


**Verifying Dataset Shape**

In [ ]:
df.shape

(100, 2)

**Importing TensorFlow and TensorFlow Hub**

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
tf.keras.backend.clear_session()

**Splitting Data into Features and Labels**

In [ ]:
X = df['review']
Y = df['sentiment']

encoder = LabelEncoder()
Y = encoder.fit_transform(Y)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

**Defining BERT Preprocessor and Encoder Layers**

In [ ]:
preprocessor = hub.KerasLayer("https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3")
encoder = hub.KerasLayer("https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-l-12-h-768-a-12/versions/4", trainable=True)

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
encoder_inputs = preprocessor(text_input)
outputs = encoder(encoder_inputs)

pooled_output = outputs["pooled_output"]

drop_out = tf.keras.layers.Dropout(0.3, name='dropout')(pooled_output)
output = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(drop_out)

model = tf.keras.Model(inputs=[text_input], outputs=[output])

**Compiling the BERT Model**

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

**Defining Early Stopping for Training**

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
batch_size = 16
epochs = 5
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[early_stopping]
)

Epoch 1/5
4/4 [==============================] - 156s 31s/step - loss: 0.8083 - accuracy: 0.5781 - val_loss: 0.6349 - val_accuracy: 0.6250
Epoch 2/5
4/4 [==============================] - 117s 30s/step - loss: 0.6224 - accuracy: 0.6719 - val_loss: 0.5475 - val_accuracy: 0.8125
Epoch 3/5
4/4 [==============================] - 116s 30s/step - loss: 0.4963 - accuracy: 0.7500 - val_loss: 0.4643 - val_accuracy: 0.8750
Epoch 4/5
4/4 [==============================] - 115s 30s/step - loss: 0.3667 - accuracy: 0.8906 - val_loss: 0.4299 - val_accuracy: 0.8750
Epoch 5/5
4/4 [==============================] - 115s 30s/step - loss: 0.2343 - accuracy: 0.9688 - val_loss: 0.4515 - val_accuracy: 0.8125
